In [12]:
import sys
sys.path.insert(0, '/home/nlp/ASR/espnet')

In [6]:
import IPython.display as ipd

sr = 16000
# ipd.display(ipd.Audio('/home/nlp/ASR/espnet/egs/FSW/mytest/test2.wav' , rate=sr))
# '/home/nlp/ASR/espnet/egs/FSW/single_wav/inference.wav'
ipd.display(ipd.Audio('/home/nlp/Demo/api/wav/output.wav' , rate=sr))
# ipd.display(ipd.Audio('/home/nlp/ASR/espnet/egs/FSW/single_wav/inference.wav',rate=sr))

In [7]:
import os
wav_path = '/home/nlp/Demo/api/wav/output.wav'
target_path = '/home/nlp/ASR/espnet/egs/FSW/single_wav/inference.wav'

if os.path.isfile(target_path):
    print('remove old inference.wav')
    os.remove(target_path)

# print('change sample rate to 16k......')
# print(os.system('ffmpeg -i {} -ar 16000 {}'.format(wav_path,target_path)))
os.system('cp {} {}'.format(wav_path , target_path))



print('get fbank .........')
print(os.system('cd /home/nlp/ASR/espnet/egs/FSW/ ; sh /home/nlp/ASR/espnet/egs/FSW/get_fbank.sh'))

# fp = open('/home/nlp/ASR/espnet/egs/FSW/single_wav/fbank/cmvn_single_wav.scp','r')
# line = fp.readline()
# fbank_path = line.split()[1].replace('\n','')
# print(fbank_path)
# fp.close()

remove old inference.wav
get fbank .........
0


In [8]:
fbank_path = '/home/nlp/ASR/espnet/egs/FSW/single_wav/fbank/dump/feats.1.ark:10'
# feats1 : /home/nlp/ASR/espnet/egs/FSW/single_wav/fbank/dump/feats.1.ark:10
# feats /home/nlp/ASR/espnet/egs/FSW/single_wav/fbank/dump/feats.1.ark:10

In [9]:
import kaldiio

fbank = kaldiio.load_mat(fbank_path)

print(fbank.shape)

(410, 83)


In [10]:
import json
import torch
import sys
sys.path.insert(0, '/home/nlp/ASR/espnet')
# from espnet.nets.pytorch_backend.e2e_asr_comformer import E2E
# 
# from espnet.nets.pytorch_backend.e2e_asr_conformer import E2E
from espnet.nets.pytorch_backend.e2e_asr_conformer_self_mix import E2E

model_dir = "/home/nlp/ASR/espnet/egs/aishell/asr1/exp/train_nodev_pytorch_conformer_train/results"

# load model
with open(model_dir + "/model.json", "r") as f:
    idim, odim, conf = json.load(f)
model = E2E.build(idim, odim, **conf)
model.load_state_dict(torch.load(model_dir + "/model.acc.best" , map_location='cpu'))
model.cpu().eval()
vocab = conf["char_list"]
# print(vocab)
# model

In [11]:
import re
from espnet.nets.beam_search import BeamSearch
bs = BeamSearch(
    scorers=model.scorers(), weights={"decoder": 0.5, "ctc": 0.5},
    sos=model.sos, eos=model.eos,
    beam_size=2, vocab_size=len(vocab))


    # GPU decoding: model.cuda(), bs.cuda()
with torch.no_grad():
    encoded = model.encode(torch.as_tensor(fbank))
    result = bs(encoded)  # get N-best results

print("N-best list:")
texts = []
for n, hyp in enumerate(result, 1):
    text = "".join(vocab[y] for y in hyp.yseq).replace("<space>", " ").replace("<eos>", "")
    scores = {k: f"{float(v):0.3f}" for k, v in hyp.scores.items()}
    print(f"{n}: {text}, score: {scores},len:{len(text)}")
    texts.append(text)
print(texts)

decoder input length: 101
max output length: 101
min output length: 0
N-best list:
1: 而是因为她也中了天价的招致招致招国招国招致招国招致招为招为招致招为招国招国招致战国招国招国招致战国招国招致国致战国交致战国券国交国券国战致国交家国致战国券国战致国招国交国致国致国致国交称于称于, score: {'decoder': '-769.010', 'ctc': '-660.644'},len:96
2: 而是因为她也中了天价的招致招致招国招国招致招国招致招为招为招致招为招国招国招致战国招国招国招致战国招国招致国致战国交致战国券国交国券国战致国交家国致战国券国战致国招国交国致国致国致国交称于称牌, score: {'decoder': '-769.238', 'ctc': '-660.468'},len:96
['而是因为她也中了天价的招致招致招国招国招致招国招致招为招为招致招为招国招国招致战国招国招国招致战国招国招致国致战国交致战国券国交国券国战致国交家国致战国券国战致国招国交国致国致国致国交称于称于', '而是因为她也中了天价的招致招致招国招国招致招国招致招为招为招致招为招国招国招致战国招国招国招致战国招国招致国致战国交致战国券国交国券国战致国交家国致战国券国战致国招国交国致国致国致国交称于称牌']


In [8]:
import json
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False   #这两行需要手动设置
import kaldiio

root = "/home/nlp/ASR/espnet/egs/aishell/asr1"
with open(root + "/dump/test/deltafalse/data.json", "r") as f:
    test_json = json.load(f)["utts"]


    
idx = 364

key, info = list(test_json.items())[idx]
print(key)
fbank = kaldiio.load_mat(info["input"][0]["feat"])

print(info)
print(info["input"][0]["feat"])

# fbank = kaldiio.load_mat('/home/nlp/ASR/espnet/egs/FSW/mfcc/raw_fbank_mytest.1.ark:6')

print(fbank.shape)

# # plot the speech feature
# plt.matshow(fbank.T[::-1])
# plt.title(key + ": " + info["output"][0]["text"])

BAC009S0764W0490
{'input': [{'feat': '/home/nlp/ASR/espnet/egs/aishell/asr1/dump/test/deltafalse/feats.1.ark:14302050', 'name': 'input1', 'shape': [410, 83]}], 'output': [{'name': 'target1', 'shape': [12, 4232], 'text': '而是因为她也中了天价的招', 'token': '而 是 因 为 她 也 中 了 天 价 的 招', 'tokenid': '2994 1694 704 36 848 58 30 66 815 124 2552 1466'}], 'utt2spk': 'S0764'}
/home/nlp/ASR/espnet/egs/aishell/asr1/dump/test/deltafalse/feats.1.ark:14302050
(410, 83)


In [9]:
# Appendix audio example in AN4
import os
import kaldiio
from IPython.display import Audio

try:
    d = os.getcwd()
    os.chdir(root)
    sr, wav = kaldiio.load_scp("data/test/wav.scp")[key]
finally:
    os.chdir(d)
Audio(wav, rate=sr)

In [10]:
import json
import torch
import sys
sys.path.insert(0, '/home/nlp/ASR/espnet')
# from espnet.nets.pytorch_backend.e2e_asr_comformer import E2E
# 
# from espnet.nets.pytorch_backend.e2e_asr_conformer import E2E
from espnet.nets.pytorch_backend.e2e_asr_conformer_self_mix import E2E

model_dir = "/home/nlp/ASR/espnet/egs/aishell/asr1/exp/train_nodev_pytorch_conformer_train/results"

# load model
with open(model_dir + "/model.json", "r") as f:
    idim, odim, conf = json.load(f)
model = E2E.build(idim, odim, **conf)
model.load_state_dict(torch.load(model_dir + "/model.acc.best" , map_location='cpu'))
model.cpu().eval()
vocab = conf["char_list"]
# print(vocab)
# model

In [11]:
import re
from espnet.nets.beam_search import BeamSearch

key, info = list(test_json.items())[idx]

# fbank = kaldiio.load_mat(info["input"][0]["feat"])
# fbank = kaldiio.load_mat('/home/nlp/ASR/espnet/egs/FSW/mfcc/raw_fbank_mytest.1.ark:6')

# setup beam search
bs = BeamSearch(
    scorers=model.scorers(), weights={"decoder": 0.5, "ctc": 0.5},
    sos=model.sos, eos=model.eos,
    beam_size=2, vocab_size=len(vocab))


# GPU decoding: model.cuda(), bs.cuda()
with torch.no_grad():
    encoded = model.encode(torch.as_tensor(fbank))
    result = bs(encoded)  # get N-best results

print("groundtruth:", info["output"][0]["text"])
print("N-best list:")
for n, hyp in enumerate(result, 1):
    text = "".join(vocab[y] for y in hyp.yseq).replace("<space>", " ").replace("<eos>", "")
    scores = {k: f"{float(v):0.3f}" for k, v in hyp.scores.items()}
    print(f"{n}: {text}, score: {scores},len:{len(text)}")

decoder input length: 101
max output length: 101
min output length: 0
groundtruth: 而是因为她也中了天价的招
N-best list:
1: 而是因为她也中了天价的招致招致招国招国招致招国招致招为招为招致招为招国招国招致战国招国招国招致战国致于招致国致战国交致战国券国交国券国战致家国家国致战国券国战致国招国交国致国致国致国交称于称于, score: {'decoder': '-772.262', 'ctc': '-657.780'},len:96
2: 而是因为她也中了天价的招致招致招国招国招致招国招致招为招为招致招为招国招国招致战国招国招国招致战国致于招致国致战国交致战国券国交国券国战致家国家国致战国券国战致国交国交国致国致国致国交称于称于, score: {'decoder': '-772.540', 'ctc': '-657.526'},len:96


In [5]:
# Attention plot
x = torch.as_tensor(fbank).unsqueeze(0)
y = result[0].yseq.unsqueeze(0)
attn = model.calculate_all_attentions(x, torch.tensor(len(fbank)), y)[0]

# plot
fig, ax = plt.subplots(2, figsize=(10, 7))
ax[0].set_title("Attention")
ax[0].matshow(attn, aspect="auto")
txt = [vocab[int(i)] for i in y[0]]
ax[0].set_yticks(range(len(txt)))
ax[0].set_yticklabels(txt)
ax[1].matshow(fbank.T[::-1], aspect="auto")
fig.tight_layout()

TypeError: iteration over a 0-d tensor

In [ ]:
# CTC posterior plot
with torch.no_grad():
    logp = model.ctc.log_softmax(encoded.unsqueeze(0))[0]
    prob = logp.exp_().numpy()

fig, ax = plt.subplots(2, figsize=(10, 7))
ax[0].set_title("CTC posterior")
vs = set(int(y) for y in result[0].yseq)
vs.add(0)
for n, i in enumerate(vs):
    v = vocab[i]
    ax[0].plot(prob[:, i], label=v, linestyle="-" if len(v) == 1 else "--")
ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax[0].set_xlim(0, len(encoded)-1)
ax[1].matshow(fbank.T[::-1], aspect="auto")
fig.tight_layout()